# DSBA 6156 - Final Project
## Predicting March Madness Results
<img src="download.jpeg" width=80, height=80/>

---

### Import dependencies

In [10]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import re

### Gather the data we are going to use from the web
Source: www.sports-reference.com

In [11]:
# # Scrape data off of the web - exported to CSV

# years = list(range(1993, 2023))
# all_data = pd.DataFrame()
# for i in range(0,len(years)):
#     table = pd.read_html(f"https://www.sports-reference.com/cbb/seasons/{years[i]}-school-stats.html",
#                      header = 1)
#     df = pd.DataFrame(table[0])
#     df = df[df['Rk'].notna()]
#     df = df[df['Rk'] != 'Rk']
#     df['School'] = df['School'].str.replace("NCAA","")
#     df['Year'] = years[i]
#     #df = df.loc[:,df.notna().any(axis=0)]
#     all_data = all_data.append(df, ignore_index=True)
    
# # Export data to CSV to avoid rerunning code every time
# all_data.to_csv("AllData1993to2022.csv", header = True)

### Read in the csv file we created and clean the data

In [17]:
bbdata = pd.read_csv("AllData1993to2022.csv")
bbdata["School"] = bbdata["School"].str.replace(u'\xa0', u' ').str.strip()
bbdata["Year"] = bbdata["Year"].astype(str)

### Read in more data from the web. This will give us the winner of the NCAA tournament each year. We will also clean this data to make sure the data is able to link to our original table properly

Source: https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_men%27s_basketball_champions

In [18]:
champs = pd.read_html("https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_men%27s_basketball_champions")
champs = champs[1].iloc[:,0:2]
champs.drop(champs[champs['Year'] == 2020].index, inplace = True)
champs["Year"] = champs["Year"].astype(str).str.strip()

### Merge the two tables together and create a new IsWinner variable. This value will be a 1 if the school is won that year, it will be a 0 if the school did not win

In [19]:
bbdata = pd.merge(bbdata, champs, how="inner", on=["Year"])
bbdata['IsWinner'] = np.where(bbdata['School'] == bbdata["Winning team"], 1, 0)

### Read in data for Naismith Player of the Year going back to 1993. If the team has the player for that year, they will be given a 1, if not, a 0

In [20]:
mvp = pd.read_html("https://en.wikipedia.org/wiki/Naismith_College_Player_of_the_Year")
mvp = mvp[2]
mvp = mvp[['Year', 'College', 'Player']]
mvp['Year'] = mvp['Year'].astype(str)
mvp = mvp.rename(columns={'College': 'NaismithCollege'})
bbdata = pd.merge(bbdata, mvp, how="left", on=["Year"])
bbdata['HasNaismithPlayer'] = np.where(bbdata['School'] == bbdata['NaismithCollege'], 1, 0)

### The below code gets the conference. Some code is missing so it will most likely not run if ran again

In [21]:
# school_urls = list(bbdata["School"].str.replace(".", "").str.lower().str.replace("&", "").str.replace(" ", "-").str.replace("(", "").str.replace(")", "").str.replace("'", ""))
# school_urls = list(map(lambda st: str.replace(st, "--", "-"), school_urls))
# school_urls = list(map(lambda st: str.replace(st, "uab", "alabama-birmingham"), school_urls))
# school_urls = list(map(lambda st: str.replace(st, "unc", "north-carolina"), school_urls))
# school_urls = list(map(lambda st: str.replace(st, "nc-state", "north-carolina-state"), school_urls))
# school_urls = list(map(lambda st: str.replace(st, "unc", "north-carolina"), school_urls))



# combined = pd.DataFrame({"URLs":school_urls, 
#                          "Year":bbdata['Year']})
# combined["final_url"] = combined["URLs"]+"/"+combined["Year"]
# combined_list = list(combined['final_url'])
# combined_list = list(map(lambda st: str.replace(st, "louisiana/", "louisiana-lafayette/"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "vmi", "virginia-military-institute"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "purdue-fort-wayne", "ipfw"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "texas-rio-grande-valley", "texas-pan-american"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "utep", "texas-el-paso"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "tcu", "texas-christian"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "utsa", "texas-san-antonio"), combined_list))
# combined_list = list(map(lambda st: str.replace(st, "uc-", "california-"), combined_list))


# all_conferences2 = []
# for i in range(0, len(final_url_list)):
#     perc_url = f"https://www.sports-reference.com/cbb/schools/{final_url_list[i]}.html"
#     r2 = requests.get(perc_url)
#     soup2 = BeautifulSoup(r2.text, 'html.parser')

#     soup = BeautifulSoup(r2.text, 'html.parser')
#     fields = soup.find_all(["p", "strong"])

#     dir_list = []
#     for f in fields:
#         dir_list.append(f.text)
#     try:
#         all_conferences2.append(dir_list[3].split(" in ",-1)[1].split(")", 1)[0])
#     except:
#         all_conferences2.append("Error")


# list2 = []
# for i in range(0, len(combined_list)):
#     list2.append(combined_list[i].split("/",-1)[0])
# list2_df = pd.DataFrame(list2, columns=["team"])
# unique = list(list2_df["team"].unique())
# combined2 = pd.DataFrame(unique, columns=["team"])
# combined2["year"] = "2022"
# combined2["url"] = combined2["team"]+"/"+combined2["year"]
# combined2["conference"] = all_conferences2
# combined2

# final_url_list = list(combined2["url"])

### Merge the conferences onto the dataset

In [23]:
confs_table = pd.read_csv("SchoolsWithConfs.csv")
confs_dict = dict(zip(confs_table['School'], confs_table['Conf']))
bbdata['Conference'] = bbdata['School'].map(confs_dict)
bbdata.to_csv("bbdatafull.csv")